In [180]:
import pandas as pd
import numpy as np

درخواست ۵:

در اینجا به دنبال آن هستیم تا با یادگیری از داده‌های موجود مدلی بسازیم که با در اختیار داشتن مشخصات و ویژگی‌های هر لپتاپ، قیمت آن در بازار را پیش‌بینی کند. برای این منظور از چند منظر و چند روش کار می‌کنیم. ابتدا داده‌هایی کنونی لپتاپ را که از سایت مقداد آی‌تی دریافت کرده‌ایم را می‌خوانیم و براساس ویژگی‌ها و قیمت‌های موجود در آن یک مدل رگرسیون را آموزش می‌دهیم. چالش اصلی این داده، تعداد محدود آن برای یادگیری مناسب مدل است. در ادامه به نحوی که در ادامه می‌آید قیمت‌های داده اصلی را حقیقی و تبدیل به قیمت معادل ۱۴۰۰ می‌کنیم و سپس با استفاده از آن داده که داده‌ای بزرگ و غنی و مربوط به خود لوتک است، مدل یادگیری دیگری می‌سازیم. 

In [181]:
megh_data = pd.read_csv('Laptops_final.csv')


In [182]:
megh_data.columns

Index(['manufactor', 'name', 'ename', 'aprice', 'sprice', 'laptop_size',
       'laptop_weight', 'laptop_type', 'Screen_size', 'Screen_type',
       'Screen_resolution', 'CPU_manufacturer', 'CPU_series', 'CPU_model',
       'CPU_frequency', 'cache_memory', 'RAM', 'RAM_type', 'hard_disk',
       'internal_memory', 'graphic_manufacturer', 'graphic_model',
       'graphic_memory', 'Laptop_id'],
      dtype='object')

In [149]:
megh_data['RAM'] = megh_data.RAM.str.split(" ",expand=True)[0]
megh_data['RAM'] = megh_data['RAM'].astype(float)
megh_data['CPU_frequency'] = megh_data.CPU_frequency.str.split(" ",expand=True)[0]
megh_data['CPU_frequency'] = megh_data['CPU_frequency'].astype(float)
megh_data['CPU_frequency'].fillna(megh_data['CPU_frequency'].notnull().mean(),inplace=True)
megh_data['hard_disk'] = megh_data.hard_disk.str.split(" ",expand=True)[0]
megh_data.drop(index=megh_data[megh_data['hard_disk']=='SATA'].index,inplace=True)
megh_data['hard_disk'] = megh_data['hard_disk'].astype(float)

megh_data['graphic_memory'] = megh_data.graphic_memory.str.split(" ",expand=True)[0]
megh_data['graphic_memory'] = megh_data['graphic_memory'].astype(float)
megh_data['graphic_memory'].fillna(megh_data['graphic_memory'].notnull().mean(),inplace=True)
megh_data['graphic_memory'].isnull().sum()


0

1-

الگوریتم رگرسیون خطی بر روی داده‌های مقداد آی‌تی:

In [150]:
train_megh_data = megh_data[['aprice','sprice','manufactor','laptop_weight','graphic_memory','CPU_manufacturer','RAM','graphic_manufacturer','Screen_resolution','Screen_size','CPU_frequency','hard_disk']]
train_megh_data = pd.get_dummies(train_megh_data,columns=['manufactor','CPU_manufacturer','Screen_resolution','graphic_manufacturer'],drop_first=True)



In [151]:
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = train_megh_data.drop(['sprice','aprice'], axis=1)
y = train_megh_data['aprice']

x_train,x_test , y_train,y_test = train_test_split(X,y, test_size=.2, random_state=313)

In [152]:
stan_scaler = StandardScaler()
x_train = stan_scaler.fit_transform(x_train)

In [153]:
model = LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

In [154]:
from sklearn.metrics import r2_score
y_pred = model.predict(stan_scaler.transform(x_test))

r2 = r2_score(y_test, y_pred)
r2 

0.005301087170150098

In [155]:
print(y_pred.mean())
print(y_pred.max())
print(y_pred.min())

print(y_test.mean())
print(y_test.max())
print(y_test.min())

42587863.493619174
77970313.92970426
21065292.139531218
41571272.62180974
215249000
10750000


الگوریتم یادگیری دیگری را اجرا می‌کنیم که این بار به جای تخمین پیوسته قیمت لپتاپ بتواند سطح قیمتی لپتاپ را در یکی از سطوح چهارگانه از ارزان تا گران تشخیص دهد. سطح‌بندی چهارگانه قیمتی بر حسب چارک اول، دوم و سوم ساخته‌ می‌شود:

In [156]:
megh_data = pd.read_csv('Laptops_final.csv')
megh_data['RAM'] = megh_data.RAM.str.split(" ",expand=True)[0]
megh_data['RAM'] = megh_data['RAM'].astype(float)
megh_data['CPU_frequency'] = megh_data.CPU_frequency.str.split(" ",expand=True)[0]
megh_data['CPU_frequency'] = megh_data['CPU_frequency'].astype(float)
megh_data['CPU_frequency'].fillna(megh_data['CPU_frequency'].notnull().mean(),inplace=True)
megh_data['hard_disk'] = megh_data.hard_disk.str.split(" ",expand=True)[0]
megh_data.drop(index=megh_data[megh_data['hard_disk']=='SATA'].index,inplace=True)
megh_data['hard_disk'] = megh_data['hard_disk'].astype(float)

In [157]:
q0, q1, q2, q3, q4 = np.quantile(megh_data.aprice,q = [0,0.25, 0.5, 0.75,1])
print(q0)
print(q1)
print(q2)
print(q3)
print(q4)

9566000.0
20525000.0
32399000.0
51900000.0
220014000.0


In [158]:
megh_data['aprice_categ'] = pd.cut(megh_data['aprice'], [q0, q1, q2, q3, q4], 
                              labels=[1, 2, 3, 4])
megh_data  = megh_data.drop('aprice',axis = 1)

In [159]:
megh_data['sprice_categ'] = pd.cut(megh_data['sprice'], [q0, q1, q2, q3, q4], 
                              labels=['min-q1', 'q1-q2', 'q2-q3', 'q3-max'])
megh_data  = megh_data.drop('sprice',axis = 1)

In [160]:
train_megh_data = megh_data[['aprice_categ','sprice_categ','manufactor','laptop_weight','CPU_manufacturer','RAM','graphic_manufacturer','laptop_type','Screen_size','Screen_resolution','CPU_frequency','hard_disk']]
train_megh_data = pd.get_dummies(train_megh_data,columns=['manufactor','CPU_manufacturer','graphic_manufacturer','laptop_type','Screen_resolution'],drop_first=True)

train_megh_data.dropna(inplace=True)
X = train_megh_data.drop(['sprice_categ','aprice_categ'], axis=1)
y = train_megh_data['aprice_categ']
x_train,x_test , y_train,y_test = train_test_split(X,y, test_size=.2, random_state=313)


In [161]:
from sklearn.neighbors import KNeighborsClassifier

class_model = KNeighborsClassifier(n_neighbors=4)
class_model.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=4)

In [162]:
from sklearn.metrics import mean_squared_error, r2_score
pred = class_model.predict(x_test)

print(pred)

[1 3 1 2 2 1 1 4 1 1 4 1 3 1 1 4 2 3 4 1 1 2 3 2 1 2 2 2 1 4 3 1 1 1 1 3 3
 4 2 4 4 4 4 1 3 4 1 3 1 1 4 4 3 4 1 1 2 3 4 3 2 3 2 4 3 4 2 1 1 2 4 4 1 1
 4 1 3 1 4 1 2 2 1 1 4 1 3 1 1 1 1 4 4 1 3 1 3 2 1 1 2 2 2 3 3 2 4 2 1 1 3
 1 3 3 3 2 4 3 1 2 1 2 1 4 3 3 3 1 4 4 4 1 3 1 1 3 1 4 4 2 3 4 3 3 1 2 4 1
 4 4 1 1 2 3 3 2 3 1 4 3 1 1 1 4 1 1 1 2 4 3 1 1 4 2 4 4 2 3 4 1 3 3 4 1 3
 1 4 1 2 3 1 2 1 2 1 2 3 4 3 3 3 4 2 1 2 2 2 4 1 4 2 2 4 3 1 1 2 1 2 3 4 2
 4 1 1 4 4 1 2 1 3 4 1 1 1 1 2 3 4 4 3 4 4 1 1 4 1 4 1 3 2 2 4 1 3 3 4 1 3
 3 1 1 1 1 3 2 2 1 1 3 2 2 3 1 1 2 2 2 3 1 4 4 1 1 4 2 1 1 2 2 2 1 2 1 4 3
 2 1 3 1 2 2 1 2 2 3 2 2 4 4 2 1 1 2 2 2 3 4 4 1 1 2 4 3 1 2 4 1 4 1 1 1 1
 4 1 1 2 1 4 1 1 3 4 4 3 3 2 1 4 2 3 1 1 1 2 1 4 2 2 4 1 1 4 4 2 2 2 2 2 1
 2 1 3 3 1 1 2 1 3 4 2 1 2 2 1 3 1 1 2 2 1 1 2 1 1 4 2 1 1 2 3 1 2 3 3 3 1
 3 1 3 2 2 2 4 2 4 1 4 4 3 3 4 1 3 1 1 1 3 3 2 2 2 1 2 1 2 1 4 1 2 1 4 2 3
 1 1 1 1 1 1 1 4 2 1 1 3 2 3 1 1 2 1 4 2 1 3 1 1 2 2 4 3 2 2 1 2 4 1 4 4 2
 2 1 2 4 3 2 3 2 1 1 2 4 

In [172]:
all_data = pd.read_csv('sales_data.csv')

In [173]:
all_data.columns

Index(['Order_ID', 'Branch', 'Order_Date', 'Order_Priority', 'Manufacturer',
       'Model_Name', 'Category', 'Screen_Size', 'Screen', 'CPU', 'RAM',
       'Storage', 'GPU', 'OS', 'OS_Version', 'Weight', 'Price', 'Quantity',
       'Discount', 'Total_Price', 'Profit', 'Ship_Duration'],
      dtype='object')

In [174]:
all_data[['year', 'month', 'day']] = all_data.Order_Date.str.split("-",expand=True)
all_data['year']=all_data['year'].astype('int')
all_data['month']=all_data['month'].astype('int')
all_data['day']=all_data['day'].astype('int')


all_data = all_data.loc[((all_data['month']<7) & (all_data['year']==1402)) | (all_data['year']<1402)]
all_data = all_data.loc[((all_data['month']>10) & (all_data['year']==1390)) | (all_data['year']>1390)]

In [175]:
model_data = all_data.groupby(['Model_Name']).sum()[['Quantity','Total_Price']].reset_index()
model_info = all_data.drop_duplicates(subset=['Model_Name'],keep='last')
model_info.drop(columns=['Order_ID','Branch','Order_Date','Order_Priority','Price','Quantity','Total_Price','Ship_Duration'],inplace=True)
model_data = pd.merge(model_info,model_data,how='inner', on='Model_Name')

/var/folders/0r/6nwh7td129301__r2brydjn00000gn/T/ipykernel_83124/1595178306.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_info.drop(columns=['Order_ID','Branch','Order_Date','Order_Priority','Price','Quantity','Total_Price','Ship_Duration'],inplace=True)


In [176]:
model_data[['Screen_size','maz1']] = model_data.Screen_Size.str.split('"',expand=True)
model_data['Screen_size']=model_data['Screen_size'].astype('float')

model_data[['RAM','maz2']] = model_data.RAM.str.split("G",expand=True)
model_data['RAM'] = model_data['RAM'].astype('float')
model_data[['laptop_weight','maz3']] = model_data.Weight.str.split("k",expand=True)
model_data['laptop_weight'] = model_data['laptop_weight'].astype('float')

model_data['CPU_manufacturer'] = model_data.CPU.str.split(" ",expand=True)[0]
model_data['CPU_manufacturer'].replace('Samsung','other',inplace=True)
model_data['CPU_manufacturer'].replace('Intel','INTEL',inplace=True)
model_data['graphic_manufacturer'] = model_data.GPU.str.split(" ",expand=True)[0]
model_data['graphic_manufacturer'].replace('ARM','Other',inplace=True)

In [178]:
model_data['Price'] = model_data['Total_Price']/model_data['Quantity']
model_data.rename(columns={'Manufacturer':'manufactor','Price':'aprice'},inplace=True)
model_data = model_data[['aprice','manufactor','laptop_weight','CPU_manufacturer','RAM','graphic_manufacturer','Screen_size']]
model_data = pd.get_dummies(model_data,columns=['manufactor','CPU_manufacturer','graphic_manufacturer'],drop_first=True)

model_data.columns



Index(['aprice', 'laptop_weight', 'RAM', 'Screen_size', 'manufactor_Apple',
       'manufactor_Asus', 'manufactor_Chuwi', 'manufactor_Dell',
       'manufactor_Fujitsu', 'manufactor_Google', 'manufactor_HP',
       'manufactor_Huawei', 'manufactor_LG', 'manufactor_Lenovo',
       'manufactor_MSI', 'manufactor_Mediacom', 'manufactor_Microsoft',
       'manufactor_Razer', 'manufactor_Samsung', 'manufactor_Toshiba',
       'manufactor_Vero', 'manufactor_Xiaomi', 'CPU_manufacturer_INTEL',
       'CPU_manufacturer_other', 'graphic_manufacturer_Intel',
       'graphic_manufacturer_Nvidia', 'graphic_manufacturer_Other'],
      dtype='object')

In [179]:
train_megh_data = megh_data[['aprice','sprice','manufactor','laptop_weight','CPU_manufacturer','RAM','graphic_manufacturer','Screen_size']]
train_megh_data = pd.get_dummies(train_megh_data,columns=['manufactor','CPU_manufacturer','graphic_manufacturer'],drop_first=True)


KeyError: "['aprice', 'sprice'] not in index"

In [ ]:
model_data.columns
model_data.drop(columns=['Model_Name','Screen','CPU','Storage','GPU','OS_Version','Profit','Weight', 'year', 'month', 'day', 'Total_Price','maz1', 'maz2','maz3' ],inplace=True)
model_data = pd.get_dummies(model_data,columns=['Manufacturer','Category','graphic_manufacturer','OS','CPU_manufacturer', 'graphic_manufacturer'],drop_first=True)


In [ ]:
megh_data['CPU_manufacturer'].replace('Carrizo','other',inplace=True)
megh_data['CPU_manufacturer'].replace('Apple','other',inplace=True)
megh_data['CPU_manufacturer'].unique()

megh_data['graphic_manufacturer'].replace('Apple','Other',inplace=True)
megh_data['graphic_manufacturer'].replace('AMD Radeon ','AMD',inplace=True)
megh_data['graphic_manufacturer'].replace('Nvidia Gforce','Nvidia',inplace=True)
megh_data['graphic_manufacturer'].replace('RTX 3060','Other',inplace=True)
megh_data['graphic_manufacturer'].replace('Geforce MX150','Other',inplace=True)
megh_data['graphic_manufacturer'].replace('GTX 1650 Ti','Other',inplace=True)
megh_data['graphic_manufacturer'].replace('2 گیگا بایت','Other',inplace=True)
megh_data['graphic_manufacturer'].unique()


In [111]:
data = pd.read_csv('sales_data.csv')
data.describe()


,Order_ID,Price,Quantity,Discount,Total_Price,Profit,Ship_Duration
count,1.017081e+06,1.017081e+06,1.017081e+06,1.017081e+06,1.017081e+06,1.017081e+06,1.017081e+06
mean,5.085400e+05,1.315689e+07,1.106288e+00,6.030857e-01,1.445983e+07,1.440988e+06,5.820368e+00
std,2.936061e+05,1.556534e+07,3.635524e-01,1.659795e+00,1.862173e+07,1.894085e+06,2.600829e+00
min,0.000000e+00,2.000000e+05,1.000000e+00,0.000000e+00,1.900000e+05,1.000000e+04,1.000000e+00
25%,2.542700e+05,3.050000e+06,1.000000e+00,0.000000e+00,3.150000e+06,3.000000e+05,4.000000e+00
50%,5.085400e+05,5.300000e+06,1.000000e+00,0.000000e+00,6.000000e+06,5.900000e+05,6.000000e+00
75%,7.628100e+05,1.905000e+07,1.000000e+00,0.000000e+00,2.015000e+07,1.970000e+06,8.000000e+00
max,1.017080e+06,3.281000e+08,7.000000e+00,9.000000e+00,8.846000e+08,7.674000e+07,1.000000e+01


In [100]:
data = pd.read_csv('sales_data.csv')
data['Manufacturer'].replace('Dell','dell',inplace=True)
data['Manufacturer'].replace('Lenovo','lenovo',inplace=True)
data['Manufacturer'].replace('Acer','acer',inplace=True)
data['Manufacturer'].replace('HP','hp',inplace=True)
data['Manufacturer'].replace('Asus','asus',inplace=True)
data['Manufacturer'].replace('MSI','msi',inplace=True)
data['Manufacturer'].replace('Apple','apple',inplace=True)
data['Manufacturer'].replace('Microsoft','microsoft',inplace=True)

data = data.loc[(data['Manufacturer']=='dell') | (data['Manufacturer']=='lenovo') | (data['Manufacturer']=='microsoft') | (data['Manufacturer']=='hp') | (data['Manufacturer']=='asus') | (data['Manufacturer']=='msi') | (data['Manufacturer']=='apple') | (data['Manufacturer']=='acer')]



In [101]:
data['Manufacturer'].unique()

array(['dell', 'lenovo', 'acer', 'hp', 'asus', 'msi', 'apple',
       'microsoft'], dtype=object)

In [102]:
megh_data['manufactor'].unique()

array(['apple', 'microsoft', 'asus', 'hp', 'msi', 'dell', 'lenovo',
       'acer'], dtype=object)

In [103]:
data[['year', 'month', 'day']] = data.Order_Date.str.split("-",expand=True)
data['year']=data['year'].astype('int')
data['month']=data['month'].astype('int')
data['day']=data['day'].astype('int')

data[['Screen_size','maz1']] = data.Screen_Size.str.split('"',expand=True)
data['Screen_size']=data['Screen_size'].astype('float')

data[['RAM','maz2']] = data.RAM.str.split("G",expand=True)
data['RAM'] = data['RAM'].astype('float')
data[['laptop_weight','maz3']] = data.Weight.str.split("k",expand=True)
data['laptop_weight'] = data['laptop_weight'].astype('float')

data['CPU_manufacturer'] = data.CPU.str.split(" ",expand=True)[0]
data['CPU_manufacturer'].replace('Samsung','other',inplace=True)
data['CPU_manufacturer'].replace('Intel','INTEL',inplace=True)
data['graphic_manufacturer'] = data.GPU.str.split(" ",expand=True)[0]
data['graphic_manufacturer'].replace('ARM','Other',inplace=True)

data.rename(columns={'Manufacturer':'manufactor'},inplace=True)


data.columns

Index(['Order_ID', 'Branch', 'Order_Date', 'Order_Priority', 'manufactor',
       'Model_Name', 'Category', 'Screen_Size', 'Screen', 'CPU', 'RAM',
       'Storage', 'GPU', 'OS', 'OS_Version', 'Weight', 'Price', 'Quantity',
       'Discount', 'Total_Price', 'Profit', 'Ship_Duration', 'year', 'month',
       'day', 'Screen_size', 'maz1', 'maz2', 'laptop_weight', 'maz3',
       'CPU_manufacturer', 'graphic_manufacturer'],
      dtype='object')

In [104]:
data['CPU_manufacturer'].unique()

array(['INTEL', 'AMD'], dtype=object)

In [105]:
megh_data['CPU_manufacturer'].unique()

array(['other', 'INTEL', 'AMD'], dtype=object)

In [98]:
data = data[['manufactor','laptop_weight','CPU_manufacturer','RAM','graphic_manufacturer']]
data = pd.get_dummies(data,columns=['manufactor','CPU_manufacturer','graphic_manufacturer'],drop_first=True)

y_prediction = model.predict(stan_scaler.transform(data))



ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- CPU_manufacturer_other
- graphic_manufacturer_Other


In [ ]:
cpi = pd.read_csv('cpi.csv')
cpi

الگوریتم یادگیری روی داده اصلی:

In [123]:
all_data = pd.read_csv('sales_data.csv')